In [1]:
# import ragas.metrics as m
import os
from dotenv import load_dotenv
import pandas as pd
from src.extractor import relationExtractor
# from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# from ragas.llms import LangchainLLMWrapper
# from ragas.embeddings import LangchainEmbeddingsWrapper

load_dotenv()
link = os.getenv('dsa_2214')
token = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = token

chapters = [
    'Data Structures and Algorithms',
    'Mathematical Preliminaries',
    'Algorithm Analysis',
    'Lists, Stacks, and Queues',
    'Binary Trees',
    'Non-Binary Trees',
    'Internal Sorting',
    'File Processing and External Sorting',
    'Searching',
    'Indexing',
    'Graphs',
    'Lists and Arrays Revisited',
    'Advanced Tree Structures',
    'Analysis Techniques',
    'Lower Bounds',
    'Patterns of Algorithms',
    'Limits to Computation',
]

<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:241: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/home/zanehutchens/uncc/research/rag/env/lib/python3.10/site-packages/pydantic/_internal/_config.py:291: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)


In [2]:
# data = pd.read_csv('data/sorting.csv')
# data.columns = ['concept', 'outcome']
data = pd.read_csv('data/sorting.csv')
data

,concept,outcome
0,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of linear search;...
1,Algorithms->Non-recursive Algorithms->Search->...,Demonstrate an understanding of binary search;...
2,Algorithms->Non-recursive Algorithms->Sort,Demonstrate an understanding of sorting;
3,Algorithms->Non-recursive Algorithms->Sort->In...,Demonstrate an understanding of insertion sort...
4,Algorithms->Non-recursive Algorithms->Sort->Se...,Demonstrate an understanding of selection sort...
5,Algorithms->Non-recursive Algorithms->Sort->Bu...,Demonstrate an understanding of bubble sort;An...
6,Algorithms->Recursive Algorithms->Recursive Al...,Demonstrate an understanding of recursion
7,Algorithms->Recursive Algorithms->Recursive Bi...,Demonstrate an understanding of recursive bina...
8,Algorithms->Recursive Algorithms->Recursive So...,Demonstrate an understanding of recursive merg...
9,Algorithms->Recursive Algorithms->Recursive So...,Demonstrate an understanding of recursive quic...


In [3]:
concept_data = data['concept'].tolist()
actual_concepts = []
for string in concept_data:
    words = string.split('->')
    for word in words:
        if word not in actual_concepts:
            actual_concepts.append(word)


outcome_data = data['outcome'].tolist()
actual_outcomes = []
for string in outcome_data:
    words = string.split(';')
    for s in words:
        if s not in actual_outcomes:
            actual_outcomes.append(s)

actual_concepts = [' '.join(actual_concepts)] * 1
# actual_outcomes = [' '.join(actual_outcomes)] * 4

In [4]:
extractor = relationExtractor(link, 
                            token, 
                            [chapters[6]], 
                            os.getenv('connection_string'), 
                            500, 
                            100, 
                            'DocumentEmbeddings', 
                            '2214_embeddings',
                            reset = True,
                            openai_model = 'gpt-4o',
                            st_model = 'all-MiniLM-L12-v2',
                            temp = 0)

In [5]:
from metrics.SemanticSimilarity import SemanticSimilarity
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, FaithfulnessMetric, ContextualRelevancyMetric

metrics = [AnswerRelevancyMetric(), FaithfulnessMetric(), ContextualPrecisionMetric(), ContextualRecallMetric(), ContextualRelevancyMetric(), SemanticSimilarity(st_model = extractor.embedding_model)]

Inside semantic similarity, st_model is all-MiniLM-L12-v2


In [6]:
concepts = extractor.identify_key_terms(10)[0]

In [ ]:
result = extractor.evaluate('terms', 5, actual_concepts, metrics)
print('-' * 20)
print(f'OPENAI MODEL: {extractor.openai_model}')
print(f'TEMPERATURE: {extractor.llm.temperature}')
print(f'SENTENCE TRANSFORMER: {extractor.embedding_model}')
print(f'TEXTBOOK: dsa_2214')
print(f'CHAPTERS TESTED: {extractor.chapters}')
print('-' * 20)
for i in result:
    print(f"METRIC: {i['name']} ----> SCORE: {i['score']}")
    print()
    print(f"REASON: {i['reason']}")
    print()
    print(f"QUERY: {i['input']}")
    print()
    print(f"OUTPUT: {i['output']}")
    print('-' * 20)

In [ ]:
full_extractor = relationExtractor(link, 
                    token, 
                    chapters, 
                    os.getenv('connection_string'), 
                    500, 
                    100, 
                    'DocumentEmbeddings', 
                    '2214_embeddings',
                    reset = True,
                    openai_model = 'gpt-4o',
                    st_model = 'all-MiniLM-L12-v2',
                    temp = 0.2)

In [ ]:
full_extractor.identify_concepts(5)
full_extractor.identify_key_terms(5)
full_extractor.identify_outcomes(5)

In [ ]:
full_extractor.identify_dependencies(full_extractor.key_terms) 

In [ ]:
full_extractor.build_knowledge_graph()

In [ ]:
full_extractor.draw_knowledge_graph()

In [ ]:
full_extractor.build_terminology()

In [ ]:
full_extractor.draw_hierarchy()

In [ ]:
full_extractor.terminology